We assume the images are located in the the `images` folder, itself located at the root folder of the project.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from data_generation import make_circle
from orientation_transformer import OrientationScoreTransformer

In [10]:
# these parameters take ~1 minute to fit to have a nice visualisation, 
# scale down the size of the image/the patch to increase speed.
img = make_circle(501)
transformer = OrientationScoreTransformer(patch_size=51, num_slices=50)
transformer.fit(img)
val = transformer.transform(img)

In [ ]:
f, axs = plt.subplots(ncols=5, figsize=(20, 5))
for ax in axs:
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
axs[0].matshow(transformer._cake_slices[0])
axs[1].matshow(transformer._wavelets[0].imag)

# zoom a little bit on the wavelet
zoomed_slice = slice(int(3*transformer.patch_size/8),int(5*transformer.patch_size/8))

axs[2].matshow(transformer._wavelets[0].imag[zoomed_slice, zoomed_slice])
axs[3].matshow(img)
axs[4].matshow(val[:, :, 0]>100)

In [ ]:
import ipyvolume
ipyvolume.quickvolshow(150*(val>80),level=[150], level_width=[10])

In [9]:
import os
import os.path
from utils import read_norm_img
import pickle

In [12]:
def transform_img_and_dump(i):
    SE2_IMAGES_PATH='../images_transformed_se2/'
    filename = f'{i:04}.pkl'
    img = read_norm_img(i)
    transformer = OrientationScoreTransformer(patch_size=51, num_slices=12)
    transformed_img = transformer.fit_transform(img)
    with open(os.path.join(SE2_IMAGES_PATH, filename), 'wb') as fh:
        pickle.dump(transformed_img, fh)

In [13]:
from joblib import Parallel, delayed

SE2_IMAGES_PATH='../images_transformed_se2/'

if not os.path.exists(SE2_IMAGES_PATH):
    os.makedirs(SE2_IMAGES_PATH)
    
Parallel(verbose=5, n_jobs=-1)(delayed(transform_img_and_dump)(i) for i in range(713, 1521))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
exception calling callback for <Future at 0x7fb419ef0a58 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
'''
Traceback (most recent call last):
  File "/home/clement/anaconda3/envs/mva/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/clement/anaconda3/envs/mva/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'orientation_transformer'
'''

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/clement/anaconda3/envs/mva/lib/python3.6/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/clement/anaconda3/envs/mva/lib/python3.6/site-pack

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [3]:

with open('../images_transformed_se2/0713.jpg', 'rb') as f:
    img = pickle.load(f)

NameError: name 'pickle' is not defined

In [2]:
plt.imshow(img[:, :, 3], cmap='gray')

NameError: name 'plt' is not defined

## Convert to jpg for disk space handling

In [9]:
PATH = '../images_transformed_se2/'

files = os.listdir(PATH)

for file in files: 
    with open(PATH + file, 'rb') as f: 
     
        img = pickle.load(f) 
        if not os.path.exists('../images_s2/' + file.rstrip('.jpg')):  
            os.makedirs('../images_s2/' + file.rstrip('.jpg')) 
        for i in range(12): 
            im = img[:, :, i] 
            plt.imsave('../images_s2/' + file.rstrip('.jpg') + '/' + '{00:02}.jpg'.format(i), im) 

In [7]:
list(range(668, 1251))[-1]

1250